# PacBio Genome Assembly contd.

We have generated an assembly for the Plasmodium falciparum chromosome from our Illumina data. Now, let’s have a look at the PacBio assembly.

## Generating pacbio assemblies

The `canu` pacbio assembly should have hopefully finished by now (check the log less canu_log.txt).

If it has not finished, you can find a pre-generated canu assembly at:

In [ ]:
ls ~/course_data/assembly/data/backup/pacbio_assemblies

Now use the `assembly-stats` script to look at the assembly statistics.

In [ ]:
assembly-stats canu-assembly/PB.contigs.fasta

How does it compare to the Illumina assembly?

Another long read assembler based on de Bruijn graphs is `wtdbg2` [https://github.com/ruanjue/wtdbg2](https://github.com/ruanjue/wtdbg2). Let’s try to build a second assembly using this assembler and compare it to the assembly produced with `canu`.

In [ ]:
wtdbg2 -t2 -i PBReads.fastq.gz -o wtdbg

In [ ]:
wtpoa-cns -t2 -i wtdbg.ctg.lay.gz -fo wtdbg.ctg.lay.fasta

In [ ]:
assembly-stats wtdbg.ctg.lay.fasta

## Comparing pacbio assemblies

How does the wtdbg2 assembly compare to the canu assembly? Both assemblies may be similar in contig number and N50, but are they really similar? Let’s map the Illumina reads to each assembly, call variants and compare.

In [ ]:
bwa index canu-assembly/PB.contigs.fasta

In [ ]:
samtools faidx canu-assembly/PB.contigs.fasta

In [ ]:
bwa mem -t2 canu-assembly/PB.contigs.fasta IT.Chr5_1.fastq IT.Chr5_2.fastq | samtools sort -@2 - | samtools mpileup -f canu-assembly/PB.contigs.fasta -ug - | bcftools call -mv > canu.vcf

Do the same for `wtdbg.ctg.lay.fasta` and then compare some basic statistics.

In [ ]:
bwa index wtdbg.ctg.lay.fasta

In [ ]:
samtools faidx wtdbg.ctg.lay.fasta

In [ ]:
bwa mem -t2 wtdbg.ctg.lay.fasta IT.Chr5_1.fastq IT.Chr5_2.fastq | samtools sort -@2 - | samtools mpileup -f wtdbg.ctg.lay.fasta -ug - | bcftools call -mv > wtdbg.vcf

In [ ]:
bcftools stats canu.vcf | grep ^SN

In [ ]:
bcftools stats wtdbg.vcf | grep ^SN

**Question:** What do you notice in terms of the number of SNP and indel calls?

The `wtdbg2` assembly has more variants due to having more errors. This is mainly due to a lack of polishing or error correction - something that the `canu` assembler performs, but the `wtdbg2` assembler does not.

## Polishing pacbio assembly

Correcting errors is an important step in making an assembly, especially from noisy long ready data. Not polishing an assembly can lead to genes not being identified due to insertion and deletion errors in the assembly sequence. To polish a genome assembly with Illumina data we use `bcftools consensus` to change homozygous differences between the assembly and the Illumina data to match the Illumina data

Run the following steps to polish the `canu` assembly

In [ ]:
bgzip -c canu.vcf > canu.vcf.gz

In [ ]:
tabix canu.vcf.gz

In [ ]:
bcftools consensus -i'QUAL>1 && (GT="AA" || GT="Aa")' -Hla -f canu-assembly/PB.contigs.fasta canu.vcf.gz > canu-assembly/PB.contigs.polished.fasta

Run the following steps to polish the `wtdbg2` assembly

In [ ]:
bgzip -c wtdbg.vcf > wtdbg.vcf.gz

In [ ]:
tabix wtdbg.vcf.gz

In [ ]:
bcftools consensus -i'QUAL>1 && (GT="AA" || GT="Aa")' -Hla -f wtdbg.ctg.lay.fasta wtdbg.vcf.gz > wtdbg.contigs.polished.fasta

Finally, align and call variants like before (bwa index/bwa mem/samtools-sort/mpileup/bcftools call) using the polished assemblies as the reference this time.

When running this analysis on these polished genomes, do we still get variants? More or less than with the raw `canu` and `wtdbg2` assemblies? Why?

Congratulations, you have reached the end of the Genome Assembly tutorial.